# Week 12 Coding Homework Notebook

This notebook is a template to help you answer the Week 12 coding quiz questions.
You can upload the datasets provided by your course to this environment and update the file paths as needed.

**Questions covered:**
1. Difference-in-differences effect for Dataset 12.1
2. Prior trends test (interaction of Group × Time) for Dataset 12.2
3. Autocorrelated errors simulation and comparison of OLS SE vs simulation SD of the coefficient
4. Variance inflation factor (VIF) for X1 in a collinearity example


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols

np.random.seed(0)
plt.rcParams['figure.figsize'] = (8, 5)


## Question 1 – DID effect in Dataset 12.1

**Prompt summary:**
- Dataset 12.1
- `Group = 1` is the treatment group, `Group = 0` is control
- `Time > 0` is the treatment period, `Time <= 0` is pre-treatment
- Outcome variable is assumed to be `Y`

We want the difference-in-differences (DID) treatment effect.

Steps:
1. Load the dataset
2. Compute average Y by group and period
3. Compute DID manually
4. (Optional) Confirm via a regression with an interaction term


In [ ]:
# === Question 1: Load Dataset 12.1 and compute DID effect ===

# TODO: Update the file name/path to match where you upload the dataset
data_path_q1 = 'homework_12.1.csv'  # change if needed

df1 = pd.read_csv(data_path_q1)
print('Data 12.1 shape:', df1.shape)
display(df1.head())

# Create indicators for treatment group and post period
df1['treat'] = (df1['Group'] == 1).astype(int)
df1['post'] = (df1['Time'] > 0).astype(int)

# 1) Manual DID using group means
group_means = (
    df1
    .groupby(['treat', 'post'])['Y']
    .mean()
    .unstack('post')
)
group_means.columns = ['pre', 'post']  # 0 = pre, 1 = post
display(group_means)

treat_change = group_means.loc[1, 'post'] - group_means.loc[1, 'pre']
control_change = group_means.loc[0, 'post'] - group_means.loc[0, 'pre']
did_effect = treat_change - control_change

print('Treatment group change:', treat_change)
print('Control group change:  ', control_change)
print('DID effect (manual):   ', did_effect)

# 2) DID via regression: Y ~ treat + post + treat:post
model_q1 = ols('Y ~ treat + post + treat:post', data=df1).fit()
print(model_q1.summary())
print('\nDID effect from regression (treat:post coefficient):', model_q1.params['treat:post'])


## Question 2 – Prior trends in Dataset 12.2

**Prompt summary:**
- Use Dataset 12.2
- Use only data *before* `Time = 0` (i.e., `Time < 0`)
- Run a linear regression:
  \( Y = \beta_0 + \beta_1 \text{Time} + \beta_2 \text{Group} + \beta_3 (\text{Group} \times \text{Time}) + \varepsilon \)
- We want the **t-value** for the interaction term `Group × Time`

Steps:
1. Load the dataset
2. Filter to `Time < 0`
3. Fit the regression with interaction
4. Extract and print the t-value of the interaction term


In [ ]:
# === Question 2: Prior trends test in Dataset 12.2 ===

# TODO: Update file name/path if needed
data_path_q2 = 'homework_12.2.csv'

df2 = pd.read_csv(data_path_q2)
print('Data 12.2 shape:', df2.shape)
display(df2.head())

# Keep only pre-treatment observations: Time < 0
df2_pre = df2[df2['Time'] < 0].copy()
print('Pre-treatment subset shape:', df2_pre.shape)

# Ensure Group is treated as numeric (0/1)
df2_pre['Group'] = df2_pre['Group'].astype(int)

# Regression: Y ~ Time + Group + Group:Time
model_q2 = ols('Y ~ Time * Group', data=df2_pre).fit()
print(model_q2.summary())

# Extract t-value for the Group:Time interaction term
t_interaction = model_q2.tvalues['Time:Group']
print('\nT-value for Group × Time interaction:', t_interaction)


## Question 3 – Autocorrelated errors simulation

**Prompt summary:**
- Generate an error series `e_t` of length 10,000
- Correlation between `e_t` and `e_{t+1}` should be about 0.8
- Mean of error ≈ 0, standard deviation ≈ 1
- Let `X = e` (the error)
- Let `Y = 2*X + new_error` (new independent error with the same properties)
- Compute:
  - OLS standard error of the `X` coefficient in a single regression
  - Simulation standard deviation of the coefficient by repeating the whole process many times

We expect the **simulation SD** of the coefficient to be larger than the naive OLS SE because of autocorrelation.


In [ ]:
# === Question 3: Autocorrelated errors and simulation ===

def generate_ar1_errors(n=10000, rho=0.8):
    """Generate AR(1) errors with correlation rho and approx sd=1."""
    eps_sd = np.sqrt(1 - rho**2)  # so that stationary variance is 1
    eps = np.random.normal(0, eps_sd, size=n)
    e = np.zeros(n)
    for t in range(1, n):
        e[t] = rho * e[t-1] + eps[t]
    return e

# Single run: check properties
n = 10000
rho_target = 0.8
e = generate_ar1_errors(n=n, rho=rho_target)
print('Mean error:', np.mean(e))
print('Std error: ', np.std(e))
print('Lag-1 correlation (e_t, e_{t+1}):', np.corrcoef(e[:-1], e[1:])[0, 1])

# Construct X and Y for the regression
X = e
new_error = generate_ar1_errors(n=n, rho=rho_target)
Y = 2 * X + new_error

X_mat = sm.add_constant(X)
ols_res = sm.OLS(Y, X_mat).fit()
beta_hat = ols_res.params[1]
se_naive = ols_res.bse[1]

print('\nSingle-run OLS estimate of beta:', beta_hat)
print('Naive OLS standard error:        ', se_naive)

# Simulation over many repetitions
R = 200  # you can increase this if you have time
beta_hats = []
for r in range(R):
    e = generate_ar1_errors(n=n, rho=rho_target)
    X = e
    new_error = generate_ar1_errors(n=n, rho=rho_target)
    Y = 2 * X + new_error
    X_mat = sm.add_constant(X)
    res = sm.OLS(Y, X_mat).fit()
    beta_hats.append(res.params[1])

beta_hats = np.array(beta_hats)
sim_sd = beta_hats.std(ddof=1)

print('\nSimulation results (R =', R, ')')
print('Mean of beta_hat over simulations:', beta_hats.mean())
print('SD of beta_hat over simulations:  ', sim_sd)
print('\nCompare: naive OLS SE vs simulation SD')
print('Naive OLS SE (single run):', se_naive)
print('Simulation SD:            ', sim_sd)


## Question 4 – Variance Inflation Factor (VIF) of X1

**Prompt summary:**
We generate:
```python
np.random.seed(0)
X1 = np.random.normal(0, 1, 1000)
X2 = np.random.normal(0, 1, 1000) + X1
X3 = np.random.normal(0, 1, 1000) + 2 * X2
```

We want the **VIF of X1**, which is:
\( \text{VIF}_{X1} = 1 / (1 - R^2) \), where \( R^2 \) is from regressing X1 on X2 and X3.


In [ ]:
# === Question 4: VIF for X1 ===

np.random.seed(0)
X1 = np.random.normal(0, 1, 1000)
X2 = np.random.normal(0, 1, 1000) + X1
X3 = np.random.normal(0, 1, 1000) + 2 * X2

df_vif = pd.DataFrame({'X1': X1, 'X2': X2, 'X3': X3})

# Regress X1 on X2 and X3
X_mat = sm.add_constant(df_vif[['X2', 'X3']])
res_vif = sm.OLS(df_vif['X1'], X_mat).fit()
R2 = res_vif.rsquared
vif_X1 = 1 / (1 - R2)

print(res_vif.summary())
print('\nR^2 for regression of X1 on X2 and X3:', R2)
print('VIF for X1:', vif_X1)
